*** 
<font color = green>

#  Home Task
</font>




In [175]:
import matplotlib.pyplot as plt
import pandas as pd 
import numpy as np 

***
<font color = green>

##  Sklearn SVM for Iris dataset
</font>



In [176]:

# YOUR_CODE.  Try linear, rbf and poly kernels 
# START_CODE 

from sklearn.datasets import load_iris
np.random.seed = 2021
iris = load_iris()
print ('data contains:',iris.keys())
X, y, labels, feature_names  = iris.data, iris.target, iris.target_names, iris['feature_names']
df_iris= pd.DataFrame(X, columns= feature_names) 
df_iris['label'] =  y
features_dict = {k:v for k,v in  enumerate(labels)}
df_iris['label_names'] = df_iris.label.apply(lambda x: features_dict[x])
df_iris

# END_CODE 

data contains: dict_keys(['data', 'target', 'frame', 'target_names', 'DESCR', 'feature_names', 'filename', 'data_module'])


,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),label,label_names
0,5.1,3.5,1.4,0.2,0,setosa
1,4.9,3.0,1.4,0.2,0,setosa
2,4.7,3.2,1.3,0.2,0,setosa
3,4.6,3.1,1.5,0.2,0,setosa
4,5.0,3.6,1.4,0.2,0,setosa
...,...,...,...,...,...,...
145,6.7,3.0,5.2,2.3,2,virginica
146,6.3,2.5,5.0,1.9,2,virginica
147,6.5,3.0,5.2,2.0,2,virginica
148,6.2,3.4,5.4,2.3,2,virginica


In [177]:
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
X_train, X_test, y_train, y_test = train_test_split(iris["data"], iris["target"], random_state = 0) 

In [178]:
scale = StandardScaler().fit(X_train)
X_train_scaled = scale.transform(X_train)
X_test_scaled = scale.transform(X_test)

<font color = green>

## Linear SVC!
</font>

In [179]:
#liner kornel
clf = SVC(kernel="linear", C=1,gamma=10).fit(X_train_scaled,y_train)
print("Linear testing")
print("train accuracy= {:.3%}".format(clf.score (X_train_scaled,y_train)))
print("test accuracy= {:.3%}".format(clf.score (X_test_scaled,y_test)))

Linear testing
train accuracy= 97.321%
test accuracy= 97.368%


<font color = green>

## SVC RBF Kernel!
</font>

In [180]:
#rbf kornel
gamma=1
clf = SVC(kernel= 'rbf', C=1, gamma=gamma).fit(X_train_scaled,y_train)
print("RBF Kernel")
print("train accuracy= {:.3%}".format(clf.score (X_train_scaled,y_train)))
print("test accuracy= {:.3%}".format(clf.score (X_test_scaled,y_test)))

RBF Kernel
train accuracy= 98.214%
test accuracy= 97.368%


<font color = green>

## SVC Poly Kernel!
</font>

In [181]:
degree= 3
clf = SVC(kernel= 'poly', C=1.0,gamma=3).fit(X_train_scaled,y_train)
print("Poly testing")
print("train accuracy= {:.3%}".format(clf.score (X_train_scaled,y_train)))
print("test accuracy= {:.3%}".format(clf.score (X_test_scaled,y_test)))

Poly testing
train accuracy= 100.000%
test accuracy= 97.368%


***
<font color = green>

##  Spam/Non-spam classification
</font>




<font color = green>

###  Problem statement
</font>

Spam filter classifies emails into spam and non-spam email. 
For this task use SVMs to build your spam filter.
This is binary classification  - whether a given email, x, is spam (y = 1) or non-spam (y = 0).
In particular, you need to convert each email into a feature vector x. 
The dataset for this task is based on a a subset of the SpamAssassin Public Corpus.
For the purpose of this exercise, it will use the body of the email (excluding the email headers)


<font color = green>

###  Preprocessing mail
</font>

Before starting on a machine learning task, it is usually insightful to take a look at examples from the dataset. Below is a sample email that contains a URL, an email address (at the end), numbers, and dollar amounts. While many emails would contain similar types of entities (e.g., numbers, other URLs, or other email addresses), the specific entities (e.g., the specific URL or specific dollar amount) will be different in almost every email. Therefore, one method often employed in processing emails is to “normalize” these values, so that all URLs are treated the same, all numbers are treated the same, etc. For example, replace each URL in the email with the unique string “http_addr” to indicate that a URL was present.
This lets the spam classifier to make a classification decision based on whether any URL was present, rather than whether a specific URL was present. This typically improves the performance of a spam classifier, since spammers often randomize the URLs, and thus the odds of seeing any particular URL again in a new piece of spam is very small.


<font color = green>

### Reveiw sample mail 
</font>



In [182]:
def get_sample(fn):
    with open(fn, 'r') as f:
        content = f.read()
    return content
    
fn=  os.path.join(path , 'emailSample1.txt')
content = get_sample(fn)
content

"> Anyone knows how much it costs to host a web portal ?\n>\nWell, it depends on how many visitors you're expecting.\nThis can be anywhere from less than 10 bucks a month to a couple of $100. \nYou should checkout http://www.rackspace.com/ or perhaps Amazon EC2 \nif youre running something big..\n\nTo unsubscribe yourself from this mailing list, send an email to:\ngroupname-unsubscribe@egroups.com\n\n"

<font color = green>

### Preprocessing and normalization steps
</font>

Preprocessing and normalization includes the following steps:
<ul>
<li> <b>Lower-casing</b>: The entire email is converted into lower case, so that captialization is ignored (e.g., IndIcaTE is treated the same as Indicate).
    
<li> <b>Stripping HTML</b>: All HTML tags are removed from the emails. Many emails often come with HTML formatting; we remove all the HTML tags, so that only the content remains.
<li> <b>Normalizing URLs</b>: All URLs are replaced with the text “httpaddr”.
<li> <b>Normalizing Email Addresses</b>: All email addresses are replaced
with the text “emailaddr”.
<li> <b>Normalizing Numbers</b>: All numbers are replaced with the text
“number”.
<li> <b>Normalizing Dollars</b>: All dollar signs ($) are replaced with the text
“dollar”.
<li> <b>Word Stemming</b>: Words are reduced to their stemmed form. For ex- ample, “discount”, “discounts”, “discounted” and “discounting” are all replaced with “discount”. Sometimes, the Stemmer actually strips o↵ additional characters from the end, so “include”, “includes”, “included”, and “including” are all replaced with “includ”.
<li> <b>Removal of non-words</b>: Non-words and punctuation have been re- moved. All white spaces (tabs, newlines, spaces) have all been trimmed to a single space character.
</ul>

<font color = green>

### Word tokenization
</font>


In [183]:
import re

In [184]:
def word_tokeniize(content):
    '''
    content: str - body of mail 
    return: list of tokens (str) e.g. ['>', 'Anyone', 'knows', 'how', 'much', 'it', 'costs', 'to', 'host', 'a']
    '''
    # YOUR_CODE.  Split the content to tokens. You may need re.split()
    # START_CODE 
    tokens = re.split( r'\s', content)

    # END_CODE 
    
    return tokens

<font color = blue >

### Check result

</font>


In [185]:
tokens  = word_tokeniize('''> Anyone knows how much it costs to host a web portal ?\n>\nWell, it depends on how many visitors you're expecting.\nThis can be anywhere from less than 10 bucks a month to a couple of $100. \nYou should checkout http://www.rackspace.com/ or perhaps Amazon EC2 \nif youre running something big..\n\nTo unsubscribe yourself from this mailing list, send an email to:\ngroupname-unsubscribe@egroups.com\n\n''')
tokens

['>',
 'Anyone',
 'knows',
 'how',
 'much',
 'it',
 'costs',
 'to',
 'host',
 'a',
 'web',
 'portal',
 '?',
 '>',
 'Well,',
 'it',
 'depends',
 'on',
 'how',
 'many',
 'visitors',
 "you're",
 'expecting.',
 'This',
 'can',
 'be',
 'anywhere',
 'from',
 'less',
 'than',
 '10',
 'bucks',
 'a',
 'month',
 'to',
 'a',
 'couple',
 'of',
 '$100.',
 '',
 'You',
 'should',
 'checkout',
 'http://www.rackspace.com/',
 'or',
 'perhaps',
 'Amazon',
 'EC2',
 '',
 'if',
 'youre',
 'running',
 'something',
 'big..',
 '',
 'To',
 'unsubscribe',
 'yourself',
 'from',
 'this',
 'mailing',
 'list,',
 'send',
 'an',
 'email',
 'to:',
 'groupname-unsubscribe@egroups.com',
 '',
 '']

<font color = blue >

### Expected output

</font>

`array(['>', 'Anyone', 'knows', 'how', 'much', 'it', 'costs', 'to', 'host',
       'a', 'web', 'portal', '?', '>', 'Well', 'it', 'depends', 'on',
       'how', 'many', 'visitors', "you're", 'expecting.', 'This', 'can',
       'be', 'anywhere', 'from', 'less', 'than', '10', 'bucks', 'a',
       'month', 'to', 'a', 'couple', 'of', '$100.', '', 'You', 'should',
       'checkout', 'http://www.rackspace.com/', 'or', 'perhaps', 'Amazon',
       'EC2', '', 'if', 'youre', 'running', 'something', 'big..', '',
       'To', 'unsubscribe', 'yourself', 'from', 'this', 'mailing', 'list',
       'send', 'an', 'email', 'to:', 'groupname-unsubscribe@egroups.com',
       '', ''], dtype='<U33')`

<font color = green>

### Lower case 
</font>


In [186]:
def lower_case(tokens):
    '''
    tokens: ndarry of str
    return: ndarry of tokens in lower case (str)
    '''
    # YOUR_CODE.  Make all tokens in lower case
    # START_CODE 
    low_tokens = np.char.lower(tokens)
    # END_CODE 
   
    return low_tokens

<font color = blue >

### Check result

</font>


In [187]:
tokens = lower_case(tokens)
tokens

array(['>', 'anyone', 'knows', 'how', 'much', 'it', 'costs', 'to', 'host',
       'a', 'web', 'portal', '?', '>', 'well,', 'it', 'depends', 'on',
       'how', 'many', 'visitors', "you're", 'expecting.', 'this', 'can',
       'be', 'anywhere', 'from', 'less', 'than', '10', 'bucks', 'a',
       'month', 'to', 'a', 'couple', 'of', '$100.', '', 'you', 'should',
       'checkout', 'http://www.rackspace.com/', 'or', 'perhaps', 'amazon',
       'ec2', '', 'if', 'youre', 'running', 'something', 'big..', '',
       'to', 'unsubscribe', 'yourself', 'from', 'this', 'mailing',
       'list,', 'send', 'an', 'email', 'to:',
       'groupname-unsubscribe@egroups.com', '', ''], dtype='<U33')

<font color = blue >

### Expected output

</font>

`array(['>', 'anyone', 'knows', 'how', 'much', 'it', 'costs', 'to', 'host',
       'a', 'web', 'portal', '?', '>', 'well', 'it', 'depends', 'on',
       'how', 'many', 'visitors', "you're", 'expecting.', 'this', 'can',
       'be', 'anywhere', 'from', 'less', 'than', '10', 'bucks', 'a',
       'month', 'to', 'a', 'couple', 'of', '$100.', '', 'you', 'should',
       'checkout', 'http://www.rackspace.com/', 'or', 'perhaps', 'amazon',
       'ec2', '', 'if', 'youre', 'running', 'something', 'big..', '',
       'to', 'unsubscribe', 'yourself', 'from', 'this', 'mailing', 'list',
       'send', 'an', 'email', 'to:', 'groupname-unsubscribe@egroups.com',
       '', ''], dtype='<U33')`

<font color = green>

### Normalize
</font>


In [188]:
def normalize_tokens (tokens):
    '''
    tokens: ndarry of str
    return: ndarry of tokens replaced with corresponding unified words
    '''
    # YOUR_CODE.  
        # Remove html and other tags
        # mark all numbers "number"
        # mark all  urls as "httpaddr"
        # mark all emails as "emailaddr"
        # replace $ as "dollar"
        # get rid of any punctuation
        # Remove any non alphanumeric characters
    #  You may  need re.sub()
    # START_CODE 
    normalized_tokens = []
    for token in tokens:
        # Remove html and other tags
        token = re.sub(r'>', '', token)

        # Mark all numbers as "number"
        token = re.sub(r'[-+]?\d+', 'number', token)

        # Mark all urls as "httpaddr"
        token = re.sub(r'\b(https?|ftp|file)://\S+', 'httpaddr', token)

        # Mark all emails as "emailaddr"
        token = re.sub(r'\S+@\S+', 'emailaddr', token)

        # Replace $ as "dollar"
        token = re.sub(r'\$', 'dollar', token)

        # Get rid of any punctuation and other non-alphanumeric characters
        token = re.sub(r'[^a-zA-Z0-9\s]', '', token)
        
        normalized_tokens.append(token)

    return normalized_tokens


<font color = blue >

### Check result

</font>


In [189]:
tokens = normalize_tokens(tokens)
tokens

['',
 'anyone',
 'knows',
 'how',
 'much',
 'it',
 'costs',
 'to',
 'host',
 'a',
 'web',
 'portal',
 '',
 '',
 'well',
 'it',
 'depends',
 'on',
 'how',
 'many',
 'visitors',
 'youre',
 'expecting',
 'this',
 'can',
 'be',
 'anywhere',
 'from',
 'less',
 'than',
 'number',
 'bucks',
 'a',
 'month',
 'to',
 'a',
 'couple',
 'of',
 'dollarnumber',
 '',
 'you',
 'should',
 'checkout',
 'httpaddr',
 'or',
 'perhaps',
 'amazon',
 'ecnumber',
 '',
 'if',
 'youre',
 'running',
 'something',
 'big',
 '',
 'to',
 'unsubscribe',
 'yourself',
 'from',
 'this',
 'mailing',
 'list',
 'send',
 'an',
 'email',
 'to',
 'emailaddr',
 '',
 '']

<font color = blue >

### Expected output

</font>

`array(['', 'anyone', 'knows', 'how', 'much', 'it', 'costs', 'to', 'host',
       'a', 'web', 'portal', '', '', 'well', 'it', 'depends', 'on', 'how',
       'many', 'visitors', 'youre', 'expecting', 'this', 'can', 'be',
       'anywhere', 'from', 'less', 'than', 'number', 'bucks', 'a',
       'month', 'to', 'a', 'couple', 'of', 'dollarnumber', '', 'you',
       'should', 'checkout', 'httpaddr', 'or', 'perhaps', 'amazon',
       'ecnumber', '', 'if', 'youre', 'running', 'something', 'big', '',
       'to', 'unsubscribe', 'yourself', 'from', 'this', 'mailing', 'list',
       'send', 'an', 'email', 'to', 'emailaddr', '', ''], dtype='<U12')`

<font color = green>

### Remove zero length tokens
</font>


In [190]:
def filter_short_tokens (tokens):
    '''
    tokens: ndarry of str
    return: ndarry of filtered tokens (str)
    '''
    original_tokens_len = len(tokens)
    
    # YOUR_CODE. Keep only tokens that lenght >0  
    # START_CODE 
    tokens = [token for token in tokens if len(token) > 0]
    
    # END_CODE     
   
    print ('Original len= {}\nRemaining len= {}'.format(original_tokens_len, len(tokens)))    
    
    return tokens


<font color = blue >

### Check result

</font>


In [191]:
tokens = filter_short_tokens(tokens)
tokens

Original len= 69
Remaining len= 61


['anyone',
 'knows',
 'how',
 'much',
 'it',
 'costs',
 'to',
 'host',
 'a',
 'web',
 'portal',
 'well',
 'it',
 'depends',
 'on',
 'how',
 'many',
 'visitors',
 'youre',
 'expecting',
 'this',
 'can',
 'be',
 'anywhere',
 'from',
 'less',
 'than',
 'number',
 'bucks',
 'a',
 'month',
 'to',
 'a',
 'couple',
 'of',
 'dollarnumber',
 'you',
 'should',
 'checkout',
 'httpaddr',
 'or',
 'perhaps',
 'amazon',
 'ecnumber',
 'if',
 'youre',
 'running',
 'something',
 'big',
 'to',
 'unsubscribe',
 'yourself',
 'from',
 'this',
 'mailing',
 'list',
 'send',
 'an',
 'email',
 'to',
 'emailaddr']

<font color = blue >

### Expected output

</font>

`Original len= 69
Remaining len= 61
array(['anyone', 'knows', 'how', 'much', 'it', 'costs', 'to', 'host', 'a',
       'web', 'portal', 'well', 'it', 'depends', 'on', 'how', 'many',
       'visitors', 'youre', 'expecting', 'this', 'can', 'be', 'anywhere',
       'from', 'less', 'than', 'number', 'bucks', 'a', 'month', 'to', 'a',
       'couple', 'of', 'dollarnumber', 'you', 'should', 'checkout',
       'httpaddr', 'or', 'perhaps', 'amazon', 'ecnumber', 'if', 'youre',
       'running', 'something', 'big', 'to', 'unsubscribe', 'yourself',
       'from', 'this', 'mailing', 'list', 'send', 'an', 'email', 'to',
       'emailaddr'], dtype='<U12')`

<font color = green>

### Stem tokens
</font>


In [192]:
from nltk.stem import PorterStemmer

In [193]:
def stem_tokens(tokens):
    '''
    tokens: ndarry of str
    return: ndarry of stemmed tokens e.g. array(['anyon', 'know', 'how', 'much', 'it', 'cost', 'to', 'host', 'a',
       'web', 'portal', 'well', 'it', 'depend', 'on', 'how', 'mani']...
    '''
    # YOUR_CODE. replace the tokens by stemmed form. You may need PorterStemmer.stem() 
    # START_CODE 
    stemmer = PorterStemmer()
    stemmed_tokens = []

    for token in tokens:
        stemmed_token = stemmer.stem(token)
        stemmed_tokens.append(stemmed_token)

    return np.array(stemmed_tokens)


<font color = blue >

### Check result

</font>


In [194]:
tokens = stem_tokens(tokens)
tokens

array(['anyon', 'know', 'how', 'much', 'it', 'cost', 'to', 'host', 'a',
       'web', 'portal', 'well', 'it', 'depend', 'on', 'how', 'mani',
       'visitor', 'your', 'expect', 'thi', 'can', 'be', 'anywher', 'from',
       'less', 'than', 'number', 'buck', 'a', 'month', 'to', 'a', 'coupl',
       'of', 'dollarnumb', 'you', 'should', 'checkout', 'httpaddr', 'or',
       'perhap', 'amazon', 'ecnumb', 'if', 'your', 'run', 'someth', 'big',
       'to', 'unsubscrib', 'yourself', 'from', 'thi', 'mail', 'list',
       'send', 'an', 'email', 'to', 'emailaddr'], dtype='<U10')

<font color = blue >

### Expected output

</font>

`array(['anyon', 'know', 'how', 'much', 'it', 'cost', 'to', 'host', 'a',
       'web', 'portal', 'well', 'it', 'depend', 'on', 'how', 'mani',
       'visitor', 'your', 'expect', 'thi', 'can', 'be', 'anywher', 'from',
       'less', 'than', 'number', 'buck', 'a', 'month', 'to', 'a', 'coupl',
       'of', 'dollarnumb', 'you', 'should', 'checkout', 'httpaddr', 'or',
       'perhap', 'amazon', 'ecnumb', 'if', 'your', 'run', 'someth', 'big',
       'to', 'unsubscrib', 'yourself', 'from', 'thi', 'mail', 'list',
       'send', 'an', 'email', 'to', 'emailaddr'], dtype='<U10')`

<font color = green>

### Vocabulary
</font>

Ususally vocabulary is built from the top most frequent tokens within all available training set.
For this task it is already prepared and provided in the `vocab.txt`

In [195]:
def get_vocabulary(fn):
    '''
    fn: str - full path to file 
    return: ndarray of str e.g. array(['aa', 'ab', 'abil', ..., 'zdnet', 'zero', 'zip'], dtype=object)
    '''
    vocab_list = pd.read_table(fn, header=None)
    vocab = np.array(vocab_list)[:,1] # first columns is index, select only words column  
    print ('len(vocab)= {:,}'.format(len(vocab)))
    return vocab

fn=  os.path.join(path , 'vocab.txt')
vocab = get_vocabulary(fn)
vocab

len(vocab)= 1,899


array(['aa', 'ab', 'abil', ..., 'zdnet', 'zero', 'zip'], dtype=object)

<font color = green>

### Feature reresentation
</font>

Every word in vocabulary is the feature of the mail - 1 is the word in in the mail and 0 otherwise. 
Thus every mail can be represented as the binary array of fixed length - number of words in vocabulary.

In [196]:
def represent_features(tokens,vocab):
    '''
    tokens: ndarry of str
    tokens: ndarry of str
    return: ndarry of binary values 1 if word from vocabulary is in mail 0 otherwise
    '''
    # YOUR_CODE. Compute the array with 1/0 corresponding to is word from vocabulary in mail 
    # START_CODE 
    tokens_represented = [1 if word in tokens else 0 for word in vocab]
    # END_CODE     

    print ('{} word(s) from vocab are in the tokens.'.format(np.sum(tokens_represented)))

   
    return tokens_represented


<font color = blue >

### Check result

</font>


In [197]:
tokens_represented = represent_features(tokens, vocab)
tokens_represented

44 word(s) from vocab are in the tokens.


C:\Users\lygov\AppData\Local\Temp\ipykernel_16272\2264587907.py:9: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  tokens_represented = [1 if word in tokens else 0 for word in vocab]


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


<font color = blue >

### Expected output

</font>

`44 word(s) from vocab are in the tokens.
array([0, 0, 0, ..., 0, 0, 0])`

<font color = green>

### Composing all steps of preprocessing 
</font>

Every word in vocabulary is the feature of the mail - 1 is the word in in the mail and 0 otherwise. 
Thus every mail can be represented as the binary array of fixed length - number of words in vocabulary.

In [198]:
def preprocess (content,vocab):
    '''
    content: str - body of mail 
    vocab: ndarray of str - list of considered words 
    '''
    # YOUR_CODE. Compute the array with 1/0 corresponding to is word from vocabulary in mail 
    # START_CODE 

    # tokenize content    
    tokens  = word_tokeniize(content)
    
    # make lower case
    tokens = lower_case(tokens)

    # normalize tokens
    tokens = normalize_tokens(tokens)

    # remove zero words
    tokens = filter_short_tokens(tokens)
    
    # stem words
    tokens = stem_tokens(tokens)
    
    # convert to binary array of features  
    tokens_represented = represent_features(tokens, vocab)
    # END_CODE     
    
    return tokens_represented

<font color = blue >

### Check result

</font>


In [199]:
preprocess (content,vocab)

Original len= 69
Remaining len= 61
44 word(s) from vocab are in the tokens.


C:\Users\lygov\AppData\Local\Temp\ipykernel_16272\2264587907.py:9: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  tokens_represented = [1 if word in tokens else 0 for word in vocab]


[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


<font color = blue >

### Expected output

</font>

`Original len= 69
Remaining len= 61
44 word(s) from vocab are in the tokens.
array([0, 0, 0, ..., 0, 0, 0])`

<font color = green>

### Training and test sets
</font>

All mails for training are preprocessed the same way as performed for sample above. 
The training set (array of feature representation) is provided in the `spamTrain.mat`
The corresponding test set is provided in the `spamTest.mat`

In [200]:
from scipy.io import loadmat

In [201]:
fn=  os.path.join(path , 'spamTrain.mat')

mat= loadmat(fn)
X_train= mat['X']
y_train= mat['y'].ravel()

print ('X_train.shape= {}',X_train.shape)
print ('y_train.shape= {}',y_train.shape)

fn=  os.path.join(path , 'spamTest.mat')
mat= loadmat(fn)
X_test = mat['Xtest']
y_test = mat['ytest'].ravel() 

print ('X_test.shape= {}',X_test.shape)
print ('y_test.shape= {}',y_test.shape)
index = 0 
print ('Sample with index  ={}: \n{}'.format(index, X_train[index]))


X_train.shape= {} (4000, 1899)
y_train.shape= {} (4000,)
X_test.shape= {} (1000, 1899)
y_test.shape= {} (1000,)
Sample with index  =0: 
[0 0 0 ... 0 0 0]


<font color = green>

### Training  the model
</font>


In [202]:
from sklearn.svm import SVC 
from sklearn.svm import LinearSVC

In [203]:
C = .1
clf= LinearSVC(C=C)
clf.fit(X_train,y_train)
print ('Score train= {}'.format(clf.score(X_train,y_train)))
print ('Score test= {}'.format(clf.score(X_test,y_test)))

Score train= 0.99975
Score test= 0.992


<font color = green>

### Determining most spam contributors
</font>


In [204]:
# print('clf.intercept_={}'.format(clf.intercept_))
# print ('clf.coef_={}'.format(clf.coef_))

# YOUR_CODE. Compute top 20 largest coeficients and return the corresponding 20 words from vocabulary
# START_CODE 

word_c = clf.coef_[0]
index = np.argsort(word_c)[-20:][::-1]

top_spam_contributors = [vocab[i] for i in index]
# END_CODE    



<font color = blue >

### Check result

</font>


In [205]:
print (top_spam_contributors)

['our', 'remov', 'click', 'basenumb', 'guarante', 'visit', 'bodi', 'will', 'numberb', 'price', 'dollar', 'nbsp', 'below', 'lo', 'most', 'send', 'dollarnumb', 'credit', 'wi', 'hour']


<font color = blue >

### Expected output

</font>

`['our' 'remov' 'click' 'basenumb' 'guarante' 'visit' 'bodi' 'will'
 'numberb' 'price' 'dollar' 'nbsp' 'below' 'lo' 'most' 'send' 'dollarnumb'
 'credit' 'wi' 'hour']`

<font color = green>

### Use model for prediction 
</font>


In [206]:
for sfn in [ 'emailSample1.txt', 'emailSample2.txt', 'spamSample1.txt', 'spamSample2.txt']:
    fn=  os.path.join(path,sfn)    
    content = get_sample(fn)
    
    # YOUR_CODE.  Preprocess the sample and get prediction 0 or 1 (1 is spam)
    # START_CODE 

    prediction = clf.predict([preprocess(content, vocab)])[0]

    # END_CODE    
    
    print ('{} is {}\n'.format(sfn, ('Not Spam','Spam')[prediction]))

print ('Latter sample:\n{1}\n{0}\n{1}'.format(content, '='*50))

Original len= 69
Remaining len= 61
44 word(s) from vocab are in the tokens.
emailSample1.txt is Not Spam

Original len= 247
Remaining len= 222
122 word(s) from vocab are in the tokens.
emailSample2.txt is Not Spam

Original len= 141
Remaining len= 97
46 word(s) from vocab are in the tokens.
spamSample1.txt is Spam

Original len= 39
Remaining len= 31
18 word(s) from vocab are in the tokens.
spamSample2.txt is Spam

Latter sample:
Best Buy Viagra Generic Online

Viagra 100mg x 60 Pills $125, Free Pills & Reorder Discount, Top Selling 100% Quality & Satisfaction guaranteed!

We accept VISA, Master & E-Check Payments, 90000+ Satisfied Customers!
http://medphysitcstech.ru





C:\Users\lygov\AppData\Local\Temp\ipykernel_16272\2264587907.py:9: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  tokens_represented = [1 if word in tokens else 0 for word in vocab]
C:\Users\lygov\AppData\Local\Temp\ipykernel_16272\2264587907.py:9: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  tokens_represented = [1 if word in tokens else 0 for word in vocab]
C:\Users\lygov\AppData\Local\Temp\ipykernel_16272\2264587907.py:9: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  tokens_represented = [1 if word in tokens else 0 for word in vocab]
C:\Users\lygov\AppData\Local\Temp\ipykernel_16272\2264587907.py:9: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  tokens_represent

<font color = blue >

### Expected output

</font>

`Original len= 69
Remaining len= 61
44 word(s) from vocab are in the tokens.
emailSample1.txt is Not Spam`

`Original len= 247
Remaining len= 222
122 word(s) from vocab are in the tokens.
emailSample2.txt is Not Spam`

`Original len= 141
Remaining len= 97
46 word(s) from vocab are in the tokens.
spamSample1.txt is Spam`

`Original len= 39
Remaining len= 31
18 word(s) from vocab are in the tokens.
spamSample2.txt is Spam`

`Latter sample:`

`==================================================`

`Best Buy Viagra Generic Online`

`Viagra 100mg x 60 Pills $125, Free Pills & Reorder Discount, Top Selling 100\% Quality & Satisfaction guaranteed!`

`We accept VISA, Master & E-Check Payments, 90000+ Satisfied Customers!
http://medphysitcstech.ru`


`==================================================`

<font color = green>

# Congratulation!
</font>

Now you know how to build Spam/Non-Spam classifier

